<a href="https://colab.research.google.com/github/userfromdarkside/DataScience/blob/main/Textprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import csv

In [ ]:
df = pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
!pip install emoji

In [ ]:
!pip install autocorrect

In [ ]:
import re
import spacy
import nltk
import emoji
from nltk.corpus import stopwords
from textblob import TextBlob
from autocorrect import Speller
from nltk.util import ngrams
from multiprocessing import pool
from collections import Counter

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Initialize necessary components
nlp = spacy.load('en_core_web_sm')
spell = Speller()
stop_words = set(stopwords.words('english'))

In [ ]:
chat_words_map = {
    "u": "you",
    "ur": "your",
    "pls": "please",
    "brb": "be right back",
    "btw": "by the way",
    "lol": "laugh out loud",
    "idk": "I don't know",
    "imo": "in my opinion",
    "smh": "shaking my head",
    "ill": "I will",
    "m": "am",
    "s": 'is',
    "gt": "greater than",
    "txt": "text",
    "r": "are",
    "lor": "laugh out loud",
    "n": "and"
}

# Custom stopwords list
custom_stopwords = ['ecommerce', 'onestop', 'product']
stop_words.update(custom_stopwords)

# Abbreviation expansion map
abbreviation_map = {
    "ASAP": "as soon as possible",
    "FYI": "for your information",
    "ETA": "estimated time of arrival",
}

# Function to preprocess text
def preprocess_text(text, use_bigrams=False, use_trigrams=False):
    # Lowercase the text
    text = text.lower()

    # Handle emojis
    text = emoji.demojize(text)

    # Tokenize text while preserving punctuation (split words from punctuation)
    tokens = re.findall(r'\w+|[^\w\s]', text)

    # Handle chat words first
    corrected_tokens = [chat_words_map.get(token, token) for token in tokens]

    # Expand abbreviations (if any)
    expanded_tokens = [abbreviation_map.get(token.upper(), token) for token in corrected_tokens]

    # Remove URLs, emails, and hashtags (after tokenizing)
    expanded_tokens = [re.sub(r'http\S+|www\S+|https\S+', '', token) for token in expanded_tokens]
    expanded_tokens = [re.sub(r'\S+@\S+', '', token) for token in expanded_tokens]
    expanded_tokens = [re.sub(r'#\w+', '', token) for token in expanded_tokens]

    # Handle negations (combine "not" with the next word)
    new_tokens = []
    i = 0
    while i < len(expanded_tokens):
        if expanded_tokens[i] == 'not' and i + 1 < len(expanded_tokens):
            new_tokens.append(expanded_tokens[i] + '_' + expanded_tokens[i + 1])
            i += 2
        else:
            new_tokens.append(expanded_tokens[i])
            i += 1

    # Rejoin tokens into a single text for punctuation removal
    text = " ".join(new_tokens)

    # Remove punctuation, numbers, and special characters AFTER chat word handling
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenization (after punctuation removal)
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    doc = nlp(" ".join(tokens))
    lemmatized_tokens = [token.lemma_ for token in doc]

    # Spell correction (optional)
    corrected_tokens = [spell(word) for word in lemmatized_tokens]

    # Generate n-grams if requested
    if use_bigrams:
        corrected_tokens = generate_ngrams(corrected_tokens, n=2)
    elif use_trigrams:
        corrected_tokens = generate_ngrams(corrected_tokens, n=3)

    # Remove named entities (optional)
    final_text = remove_named_entities(" ".join(corrected_tokens))

    return final_text

# Function to remove named entities
def remove_named_entities(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.ent_type_]
    return " ".join(tokens)

In [ ]:
df['cleaned_message'] = df['Message'].apply(lambda x: preprocess_text(x, use_bigrams=False, use_trigrams=False))

In [ ]:
df['cleaned_message'].head()

,cleaned_message
0,go during point crazy available bugs great wor...
1,ok lar joke if oni
2,free entry only comp win fa cup final may text...
3,say early hor c already say
4,nah think go usf live around though


In [ ]:
def word_counts(column):
  counter = Counter()
  all_text = ' '.join(column)
  words = all_text.split()
  counter.update(words)
  return counter.most_common()
word_count_list = word_counts(df['cleaned_message'])
word_count_list[:50]

[('get', 701),
 ('call', 677),
 ('go', 588),
 ('great', 423),
 ('text', 393),
 ('not', 379),
 ('come', 313),
 ('lt', 311),
 ('know', 291),
 ('good', 288),
 ('ok', 287),
 ('free', 279),
 ('please', 271),
 ('send', 265),
 ('want', 259),
 ('laugh', 254),
 ('love', 248),
 ('like', 247),
 ('do', 241),
 ('loud', 239),
 ('time', 236),
 ('say', 232),
 ('day', 213),
 ('see', 209),
 ('think', 207),
 ('tell', 206),
 ('take', 195),
 ('need', 187),
 ('make', 179),
 ('well', 172),
 ('reply', 171),
 ('stop', 168),
 ('home', 167),
 ('still', 161),
 ('sorry', 159),
 ('back', 155),
 ('p', 146),
 ('phone', 145),
 ('hi', 140),
 ('msg', 138),
 ('find', 135),
 ('ask', 133),
 ('give', 133),
 ('later', 133),
 ('work', 132),
 ('miss', 131),
 ('mobile', 129),
 ('right', 128),
 ('hope', 127),
 ('c', 124)]

In [ ]:
slang_words = ['u', 'ur', 'pls', 'brb', 'btw', 'lol', 'idk', 'imo', 'smh', 'ill','m',
               's','gt','txt','r','lor','n']
for row in df['cleaned_message']:
  words = row.split()
  if any(word in slang_words for word in words):
    print(row)

england v do not miss goal team news text national team eg england try male scotland txt poboxoxwwq
hi babe I m home wan na something xx
that s cool gentleman treat dignity respect
want get lay want real dog location send direct mob join uk large dog network bt txt gravel not
what s staff name take class we
hi baby I m cruising girl friend give call hour home that s alright font font love jenny xxx
sm ac blind date u road aberdeen check http img sms blind date send hide
cup tell ppl i m gone b bit l cos bus hav gon past cos full I m still waiting pete x
fancy sha interested sextextuk com text uk szy txt cost per msg tns website x
teach ranjith sir call sms like because he s verifying project probe tell pa do not mistake
message text miss sender name miss number miss send date miss lot that s everything miss send com
what s do not want sleep
wnt buy urgently vry urgent hv shortage lt great lac source ang dis amt lt great lac that s pro
want get lay want real dog location send direct mob

In [ ]:
# Encode the label column
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Category'])

In [ ]:
df.columns

Index(['Category', 'Message', 'cleaned_message'], dtype='object')

In [ ]:
y.shape

(5572,)

In [ ]:
# Extract features using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['cleaned_message'])

In [ ]:
# Split the train_set and the test_set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15,random_state=42,shuffle=True)

In [ ]:
# Let's using Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [ ]:
# Let's evaluate it
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print('train_set_score:',accuracy_score(y_train,model.predict(X_train)))
print('test_set_score:',accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       729
           1       1.00      0.75      0.86       107

    accuracy                           0.97       836
   macro avg       0.98      0.87      0.92       836
weighted avg       0.97      0.97      0.97       836

[[729   0]
 [ 27  80]]
train_set_score: 0.9742398648648649
test_set_score: 0.9677033492822966


In [ ]:
# why the slang words are still over there. needed to review the preprocessed_text again

